# Compute curvature directly on population level

In [3]:
%matplotlib inline
import cortex
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import h5py

#### Load language data

In [4]:
path = '/Users/jiamingxu/Desktop/Language_straightenting/data/language/AA_wheretheressmoke.hf5'
with h5py.File(path, 'r') as file:
    file_name = list(file.keys())[0]
    response_trials = np.array(file[file_name])
    # get rid of the first 50 TRs (due to onset effect in AC)
    response_trials = response_trials[:, 50:, :]
    print(response_trials.shape) 

subject, xfm = 'AA', '20180905AA-sg-auto'
mask = cortex.db.get_mask(subject, xfm)

(10, 241, 95556)


#### Get ROIs

In [5]:
# choose functional roi 
#f_rois = ['V1','hMT','FFA']  
f_rois = ['AC','sPMv']
f_roi_voxs = {} # get indices of roi voxels in cortical map
# get 3d mask of voxels that belong to roi
roi_masks = cortex.utils.get_roi_masks(subject, xfm, roi_list=f_rois, gm_sampler='cortical', split_lr=False, threshold=None, return_dict=True)
for roi in f_rois:
    roi_mask = roi_masks[roi]
    f_roi_voxs[roi] = np.where(roi_mask[np.where(mask)])[0]

Cutting 0 overlapping voxels (should be < ~50)


In [6]:
# choose anatomical roi (PFC & precuneus)
%cd 'data/rois'
a_roi = ['parsopercularis','parstriangularis','superiorfrontal','rostralmiddlefrontal','caudalmiddlefrontal','frontalpole','precuneus']
roi_data = np.load(f'{subject}_roi.npy', allow_pickle=True).item()
a_roi_voxs = {}
for roi in a_roi:
    a_roi_voxs[roi] = roi_data[roi]
    
# combine PFC rois
rois_to_combine = [
    'parsopercularis',
    'parstriangularis',
    'superiorfrontal',
    'rostralmiddlefrontal',
    'caudalmiddlefrontal',
    'frontalpole'
]
pfc_voxs = []
for roi in rois_to_combine:
    pfc_voxs.extend(a_roi_voxs[roi])
    
pfc_voxs = list(set(pfc_voxs)) # remove potential duplicates

a_roi_voxs['prefrontal'] = pfc_voxs # update dict 

for roi in rois_to_combine: # remove old keys
    del a_roi_voxs[roi]


# combine functional and anatomical
roi_voxs = {**f_roi_voxs, **a_roi_voxs}  

# print number of voxels in each ROI
len(roi_voxs)
for key, value in roi_voxs.items():
    print(f"{key}: {len(value)} items")

/Users/jiamingxu/Desktop/Language_straightenting/data/rois
AC: 2124 items
sPMv: 271 items
precuneus: 2845 items
prefrontal: 15222 items


/Users/jiamingxu/miniconda3/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


#### Compute curvature

In [9]:
# compute (average) curvature from a set of observations
def curvature(X):
    V = X[1:] - X[:-1]
    norms = np.sqrt((V**2).sum(axis=1))
    dots = (V[1:] * V[:-1]).sum(axis=1)
    coss = dots / (norms[1:] * norms[:-1])
    #print(coss)
    angles = np.arccos(coss)
    return angles.mean()

In [13]:
mean_signals = np.mean(response_trials, axis = 0)

# this dict stores roi: a 2d array (timepoints x voxels) 
roi_data = {}

# number of voxels per ROI (dimensionality) 
n_voxels = 2000

for roi in roi_voxs:
    voxel_indices = roi_voxs[roi]  
    
    if len(voxel_indices) < n_voxels:
        selected_voxels = np.random.choice(voxel_indices, n_voxels, replace=True)
    else:
        selected_voxels = np.random.choice(voxel_indices, n_voxels, replace=False)
    
    roi_voxel_data = mean_signals[:, selected_voxels]  # shape: (timepoints x n_voxels)
    roi_data[roi] = roi_voxel_data

for roi in roi_data:
    curv = np.degrees(curvature(roi_data[roi]))
    print(f"Average Curvature for {roi}: {curv}")

Average Curvature for AC: 105.30823970921482
Average Curvature for sPMv: 106.7915441391399
Average Curvature for precuneus: 109.14056376475351
Average Curvature for prefrontal: 110.6685390969067
